In [1]:
!pip install SpeechRecognition

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install gtts

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install playsound

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install pyttsx3

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip install comtypes

Defaulting to user installation because normal site-packages is not writeable


In [7]:
!pip install pypiwin32

Defaulting to user installation because normal site-packages is not writeable


In [8]:
!pip install librosa pyttsx3

Defaulting to user installation because normal site-packages is not writeable


In [9]:
!pip uninstall pyttsx3 -y
!pip install pyttsx3

Found existing installation: pyttsx3 2.98
Uninstalling pyttsx3-2.98:


ERROR: Exception:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\shutil.py", line 825, in move
    os.rename(src, real_dst)
PermissionError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\pyttsx3-2.98.dist-info\\' -> 'C:\\Users\\YOGESH GOWDA\\AppData\\Local\\Temp\\pip-uninstall-t5pvfqh_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\pip\_internal\cli\base_command.py", line 180, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\pip\_internal\commands\uninstall.py", line 105, in run
    uninstall_pathset = req.uninstall(
                        ^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\pip\_internal\req\req_install.py", line 687, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "C:\ProgramData\

Defaulting to user installation because normal site-packages is not writeable


In [10]:
!pip install librosa pyttsx3 numpy

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import librosa
import numpy as np
import pyttsx3
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
import speech_recognition as sr

# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Function to play the response using pyttsx3
def speak(text):
    print("Assistant:", text)
    engine.say(text)
    engine.runAndWait()

# Load a pre-trained RNN model for emotion classification
def load_rnn_model():
    model = Sequential([
        SimpleRNN(128, input_shape=(None, 13), return_sequences=True),
        Dropout(0.2),
        SimpleRNN(64),
        Dense(4, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Normally, you would load weights from a pre-trained model
    # model.load_weights('path_to_pretrained_weights.h5')
    
    return model

rnn_model = load_rnn_model()

# Function to analyze emotion using RMS energy
def analyze_emotion_rms(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None
    
    rms = librosa.feature.rms(y=y).mean()
    print(f"RMS Energy: {rms}")
    
    # Simple logic for RMS-based emotion detection
    if rms < 0.01:
        return "sad"
    elif 0.01 <= rms < 0.02:
        return "neutral"
    elif 0.02 <= rms < 0.04:
        return "happy"
    else:
        return "angry"

# Function to analyze speech-to-text and punctuation
def analyze_emotion_from_speech_to_text(file_path):
    recognizer = sr.Recognizer()
    try:
        with sr.AudioFile(file_path) as source:
            audio = recognizer.record(source)
            text = recognizer.recognize_google(audio)
            print(f"Transcribed Text: {text}")
            
            # Punctuation-based emotion detection
            if "!" in text:
                return "angry"
            elif "?" in text:
                return "confused"
            elif "." in text:
                return "neutral"
            else:
                return "happy"
    except Exception as e:
        print(f"Speech-to-text error: {e}")
        return None

# Function to extract MFCC features and make predictions using RNN
def analyze_emotion_from_audio(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None
    
    # Extract MFCC features
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs_padded = pad_sequences([mfccs.T], maxlen=200, padding='post', dtype='float32')
    
    # Predict emotion using RNN
    prediction = rnn_model.predict(mfccs_padded)
    emotion_index = np.argmax(prediction)
    
    # Map prediction index to emotion
    emotion_mapping = {0: "happy", 1: "sad", 2: "angry", 3: "neutral"}
    emotion = emotion_mapping.get(emotion_index, "neutral")
    return emotion

# Function to generate a response based on emotion
def generate_emotion_response(emotion):
    if emotion == "happy":
        return "Hey, you sound happy! How can I assist you further?"
    elif emotion == "sad":
        return "Oh, why are you feeling sad? I'm here for you."
    elif emotion == "angry":
        return "Why are you feeling angry? Let me know if there's anything I can do."
    elif emotion == "confused":
        return "You sound confused. Let me help you out."
    elif emotion == "neutral":
        return "Hello there! You sound neutral. How may I assist you today?"

# Main function to simulate assistant interaction
def main():
    print("Voice Assistant Simulation (using pre-recorded audio files)")
    
    while True:
        audio_file = input("Enter the path of the audio file (or type 'exit' to quit): ")
        if audio_file.lower() == "exit":
            speak("Goodbye! Have a nice day.")
            break
        
        print("Analyzing emotion using RMS...")
        emotion_rms = analyze_emotion_rms(audio_file)
        print("Analyzing emotion using RNN...")
        emotion_rnn = analyze_emotion_from_audio(audio_file)
        print("Analyzing emotion using Speech-to-Text...")
        emotion_speech_to_text = analyze_emotion_from_speech_to_text(audio_file)
        
        # Choose the first valid emotion detected
        emotion = emotion_rms or emotion_rnn or emotion_speech_to_text
        if emotion:
            response = generate_emotion_response(emotion)
            speak(response)
        
        load_request = input("Do you want to load data? (yes/no): ")
        if load_request.lower() == "yes":
            data_file = input("Enter the data file path (or press Enter for default 'data.txt'): ")
            if not data_file:
                data_file = "data.txt"
            load_data(data_file)

main()
